# Chapter 9

Let's take a little break after a long chapter about pairs and before another long chapter about 3 essential types of Michelson: maps, big maps and sets.

In this chapter, we will learn more about less common operations that you can accomplish in a Michelson smart contract. First of all, we will check how to convert types between each other, which is called "casting". Then, we will have a look at the different possibilities to compare values and we will finish with a talk about bytes and their use cases.

## Casting types

One particularly pesky headache of using a strongly typed language is the conversion of values from one type to another. Obviously, some conversions are forbidden or impossible, it wouldn't make any sense to convert an integer to a list. However, some conversions may seem practical: after all, a value that's an integer, a natural number or a mutez is a numeric value and in some situations, it could be more convenient to change its type. 

Imagine a contract with a storage that's a value of type `nat`, you receive 2 ints, you add them, get a new `int` and want to save it in the storage. But you can't, because the storage needs a `nat` value. Or vice-versa, the storage is an `int` value, you add 2 nats together and want to get an `int` back. In this situations, Michelson offers two instructions: **`ABS`** will return the absolute value of a number (i.e a `nat` value) and **`INT`** will change a `nat` value into an `int`.

In [2]:
storage int ;
parameter nat ;
code {
    CAR ;
    INT ;
    NIL operation ;
    PAIR
} ;

RUN %default 9 8 ;

storage int;
parameter nat;
code { CAR ; INT ; NIL operation ; PAIR };
RUN: use %default; drop all; push (9, 8);
CAR: pop (9, 8); push 9;
INT: pop 9; push 9;
NIL: push [];
PAIR: pop [], 9; push ([], 9);

value,type
9,int


In [3]:
storage nat ;
parameter int ;
code {
    CAR ;
    ABS ;
    NIL operation ;
    PAIR
} ;

RUN %default 9 8 ;

storage nat;
parameter int;
code { CAR ; ABS ; NIL operation ; PAIR };
RUN: use %default; drop all; push (9, 8);
CAR: pop (9, 8); push 9;
ABS: pop 9; push 9;
NIL: push [];
PAIR: pop [], 9; push ([], 9);

value,type
9,nat


You may also want to check if the `int` value you have in the stack is above zero before turning it into a `nat` value (because **`ABS`** will remove the negative sign of any integer). If it's the case, you can use **`ISNAT`**. If the `int` value is equal or greater than zero, the instruction returns `(option nat) (Some value)`. If the `int` value is less than zero, the instruction returns `(option nat) (None)`:

In [6]:
storage nat ;
parameter int ;
code {
    CAR ;
    ISNAT ;
    IF_SOME
        {
            NIL operation ;
            PAIR
        }
        { FAIL }
} ;

RUN %default 9 8 ;

storage nat;
parameter int;
code { CAR ; ISNAT ; { IF_NONE { { UNIT ; FAILWITH } } { NIL operation ; PAIR } } };
RUN: use %default; drop all; push (9, 8);
CAR: pop (9, 8); push 9;
ISNAT: pop 9; push (9,);
IF_NONE: pop (9,); push 9;
NIL: push [];
PAIR: pop [], 9; push ([], 9);

value,type
9,nat


Now if you change `9` for `-9`, you will see the contract fail as the negative integer triggers the **`FAIL`** instruction.

You can also use "hacks" to cast some values. One of the most frequently encountered is a hack based on the subtraction. Let's say you need to cast a `timestamp` into an `int`. There is no instruction for that. However, if you subtract a `timestamp` from another `timestamp`, you get an `int`. You can then create a `timestamp` equal to zero and subtract it from the `timestamp` you want to cast into an `int`:

In [30]:
storage int ;
parameter unit ;
code {
    DROP ;
    PUSH timestamp 0 ;
    NOW ;
    SUB ;
    NIL operation ;
    PAIR ;
} ;

RUN %default Unit 0 ;

storage int;
parameter unit;
code { DROP ; PUSH timestamp 0 ; NOW ; SUB ; NIL operation ; PAIR };
RUN: use %default; drop all; push (Unit, 0);
DROP: pop (Unit, 0);
PUSH: push 0;
NOW: push 1594026174;
SUB: pop 1594026174, 0; push 1594026174;
NIL: push [];
PAIR: pop [], 1594026174; push ([], 1594026174);

value,type
1594026174,int


Now you have an `int` whose value is the same as the provided `timestamp`!

With a little bit of imagination, it is also possible to cast other values, for example a `mutez` into a `nat`! This time, we can use **`EDIV`** to turn a `mutez` value into an `int` value. When you divide two `mutez` values together, you get an optional value with a pair that contains a `nat` (the result) in the left field. If you divide a value by `1`, the result will be the same value, then if you divide `mutez 22` by `mutez 1`, the result will be `nat 22` as illustrated below:

In [32]:
storage nat ;
parameter unit ;
code {
    DROP ;
    PUSH mutez 1 ;
    PUSH mutez 5566778899 ;
    EDIV ;
    IF_SOME 
        {
            CAR ;
            NIL operation ;
            PAIR ;
        }
        { FAIL }
} ;

RUN %default Unit 0 ;

storage nat;
parameter unit;
code { DROP ; PUSH mutez 1 ; PUSH mutez 5566778899 ; EDIV ; { IF_NONE { { UNIT ; FAILWITH } } { CAR ; NIL operation ; PAIR } } };
RUN: use %default; drop all; push (Unit, 0);
DROP: pop (Unit, 0);
PUSH: push 1;
PUSH: push 5566778899;
EDIV: pop 5566778899, 1; push ((5566778899, 0),);
IF_NONE: pop ((5566778899, 0),); push (5566778899, 0);
CAR: pop (5566778899, 0); push 5566778899;
NIL: push [];
PAIR: pop [], 5566778899; push ([], 5566778899);

value,type
5566778899,nat
